In [ ]:
from Helpers import *
from Controller import *

In [ ]:
def f(x,u):
    g = 9.81
    L = .5
    #return np.array([x[1],-g/L*x[0]+u[0]])
    return np.array([x[1],-g/L*sin(x[0])+u[0]])

def fx(x,u):
    g = 9.81
    L = .5
    return np.array([[0,1],[-g/L*cos(x[0]),0]])
    #return np.array([[0,1],[-g/L,0]])

def fu(x,u):
    return np.array([0,1]).reshape((2,1))

def l(x,u,r1):
    return r1*u[0]**2

def lx(x,u):
    return np.zeros(2)

def lu(x,u,r1):
    return r1*2*u[0]

def lxx(x,u):
    return np.zeros((2,2))

def luu(x,u,r1):
    return r1*2

def h(x,w1,w2,xtarg):
    return w1*((x[0]-xtarg)*(x[0]-xtarg)) + w2*x[1]*x[1]

def hx(x,w1,w2,xtarg):
    return np.array([w1*2*(x[0]-xtarg),w2*2*x[1]])

def hxx(x,w1,w2):
    return np.array([[w1*2,0],[0,w2*2]])

In [ ]:
def plotLQGvsILQG(xLQG,uLQG,xILQG,uILQG,JLQG,JILQG,Duration,xtarg,K):
    fig,ax = plt.subplots(2,figsize = (8,10))
    ax[0].plot(np.linspace(0,Duration,K),xLQG[:,0]/(2*pi)*360,label = "LQG")
    ax[0].plot(np.linspace(0,Duration,K),xILQG[:,0]/(2*pi)*360,label = "ILQG")
    ax[0].plot(np.linspace(0,Duration,K),np.ones(K)*xtarg/(2*pi)*360,color = "black", linestyle = "--",alpha = .5)
    ax[0].set_xlabel("Time [sec]")
    ax[0].set_ylabel("Angle [Degree]")
    ax[0].set_ylim(-90,90)
    ax[0].set_title("ILQG Trajectory on Single-Arm Pendulum")
    ax[0].legend()

    ax[1].plot(np.linspace(0,Duration,K)[:-1],uLQG[:,0],label = "LQG")
    ax[1].plot(np.linspace(0,Duration,K)[:-1],uILQG[:,0],label = "ILQG")
    ax[1].set_xlabel("Time [sec]")
    ax[1].set_ylabel("Command (Nm)")
    ax[1].legend()
    ax[1].set_title("$J_{LQG}$ = "+str(JLQG)[:6]+"]" + "$,J_{ILQG}$ = "+str(JILQG)[:6]+"]")
    return ax

def ComputeCost(x,u,w1,w2,r1,targ):
    pos = x[-1,0]
    vel = x[-1,1]
    J = 0
    for k in range(K-1):
        J+= r1*u[k]*u[k]
    J+= w1*(pos-targ)*(pos-targ)+w2*vel*vel
    return J

In [ ]:
w1 = 1e6
w2 = 1e4
r1 = 1e-3
Duration = 2
xtarg = -pi
K = 6000
xILQG,uILQG = ILQG(Duration,w1,w2,r1,xtarg,K)
xLQG,uLQG = SingleArmLQG(Duration,w1,w2,r1,xtarg,K)
JLQG = ComputeCost(xLQG,uLQG,w1,w2,r1,xtarg)
JILQG = ComputeCost(xILQG,uILQG,w1,w2,r1,xtarg)
ax = plotLQGvsILQG(xLQG,uLQG,xILQG,uILQG,JLQG,JILQG,Duration,xtarg,K)
print("The Total cost of the LQG is "+str(JLQG))
print("The Total cost of the ILQG is "+str(JILQG))
ax[0].set_ylim(-180,180)